## How to narrow out of there are only songs in English

In [35]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import argparse
import logging
import spotipy
import pandas as pd
import numpy as np
import requests
from spotipy import oauth2
import re
import random

import credentials

In [36]:
# Gain access to mainupation
SCOPE = ('user-read-recently-played,user-top-read,user-library-read,user-read-currently-playing,playlist-read-private,playlist-modify-private,playlist-modify-public,user-read-email,user-modify-playback-state,user-read-private,user-read-playback-state')
sp_oauth = oauth2.SpotifyOAuth(credentials.SPOTIPY_CLIENT_ID,credentials.SPOTIPY_CLIENT_SECRET, credentials.SPOTIPY_REDIRECT_URI ,scope=SCOPE )

#click "Accept" in your browser when the auth window pops up
code = sp_oauth.get_auth_response(open_browser=True)
token = sp_oauth.get_access_token(code)
refresh_token = token['refresh_token']
sp = spotipy.Spotify(auth=token['access_token'])
username = sp.current_user()['id']

/var/folders/pz/lc5f5cgd1m5_ghmjddlz8r600000gn/T/ipykernel_49363/3544539949.py:7: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = sp_oauth.get_access_token(code)


In [37]:
def find_and_filter(settings, genres_list, sp):
    #FINDING
    # Generate a list of 10 track IDs for each genre
    track_ids = [track['id'] for genre in genres_list for track in sp.search(q='genre:' + genre, type='track', limit=10)['tracks']['items']]

    # Shuffle the list of track IDs
    random.shuffle(track_ids)

    # Generate recommendations based on the shuffled track IDs
    tracks = [track for i in range(0, len(track_ids), 5) for track in sp.recommendations(seed_genre=genres_list, seed_tracks=track_ids[i:i+5], limit=100)['tracks']]
    track_ids = []
    for track in tracks:
        track_ids.append(track['uri'])
    print("got reccomendations")
    print(len(tracks))

    song_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    song_df = pd.DataFrame(columns = song_features_list, index=range(len(track_ids)))

    # Fill all cells with NaN
    song_df[:] = float('nan')

    # Create empty dict
    playlist_features = {}
    playlist_features["artist"] = []
    playlist_features["album"] = []
    playlist_features["track_name"] = []
    playlist_features["track_id"] = []
    print('here 1')

    for i in range(len(track_ids)):  
        #song_df["artist"][i] = tracks[i]["album"]["artists"][0]["name"]
        playlist_features["artist"].append(tracks[i]["album"]["artists"][0]["name"])
        playlist_features["album"].append(tracks[i]["album"]["name"])
        playlist_features["track_name"].append(tracks[i]["name"])
        playlist_features["track_id"].append(tracks[i]["id"])
    print('here 2')
    song_df["artist"] =  playlist_features["artist"]
    song_df["album"] =  playlist_features["album"]
    song_df["track_name"] =  playlist_features["track_name"]
    song_df["track_id"] =  playlist_features["track_id"]
    print('here 3')
    for i in range(0, len(track_ids), 100):  
        audio_features = sp.audio_features(track_ids[i:i+100])# Batch size of 100 for API requests
        for j in range(len(audio_features)):
            # Update the dataframe with information from the dictionary for the specific track_id
            track_id = audio_features[j]["id"]
            dict_info = {}
            for feature in song_features_list[4:]:
                dict_info[feature] = audio_features[j][feature]
            for key, value in dict_info.items():
                song_df.loc[song_df['track_id'] == track_id, key] = value
    #FILTERING
    for index, setting in settings.iterrows():
        if setting["On"]:
            level = int(setting["Level"])/50 
            var = 0.1
            mean_1 = 0.5
            mean_2 = song_df[setting["Name"]].mean()
            if setting["Name"] == "tempo":
                mean_1 = 90 # kinda just guessing on this one
                var = song_df[setting["Name"]].var()
                song_df_1 = song_df[(song_df[setting["Name"]] >= level*mean_1-3*var) & (song_df[setting["Name"]] <= level*mean_1+3*var)]
                song_df_2 = song_df[(song_df[setting["Name"]] >= level*mean_2-3*var) & (song_df[setting["Name"]] <= level*mean_2+3*var)]
            else:
              song_df_1 = song_df[(song_df[setting["Name"]] >= level*mean_1-1.5*var) & (song_df[setting["Name"]] <= level*mean_1+1.5*var)]
              song_df_2 = song_df[(song_df[setting["Name"]] >= level*mean_2-1.5*var) & (song_df[setting["Name"]] <= level*mean_2+1.5*var)]
            song_df =  pd.concat([song_df_1, song_df_2], ignore_index = True)
    song_df.drop_duplicates(keep='first', inplace =  True)
    print(len(song_df))
    print('here 226')
    print("found "+str(len(song_df)))
    return song_df
def gather_songs(sp, generes_list, settings_df, goal):
    song_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    song_df = pd.DataFrame(columns = song_features_list)
    final_df = find_and_filter(settings_df,generes_list,sp)
    while (len(final_df) < goal):
        print("Still under. At length : ")
        print(len(final_df))
        getter = find_and_filter(settings_df,generes_list,sp)
        final_df = pd.concat([final_df, getter], ignore_index = True)
        final_df.drop_duplicates(keep='first', inplace =  True)
    if len(final_df) > 100:
      final_df = final_df[:100]
    return final_df

In [9]:
settings = [{"Name": "danceability", "On": False, "Level": 73},
            {"Name": "energy", "On": False,"Level": 88},
            {"Name": "valence", "On": False,"Level": 0},
            {"Name": "loudness","On": False, "Level": 100},
            {"Name": "instrumentalness","On": False, "Level": 22},
            {"Name": "liveness", "On": False,"Level": 10}]

settings_df = pd.DataFrame(settings)
goal = 5
generes_list = ["pop","indie-pop","indie"]

songs = gather_songs(sp,generes_list,settings_df, goal)

got reccomendations
600
here 1
here 2
here 3
519
here 226
found 519


In [26]:
song_ex = songs.iloc[0]
print(song_ex)

artist                         Justine Skye
album                             Dark Side
track_name          Collide (Sped Up Remix)
track_id             2p6Fh5ruk8YUaO43huubHW
danceability                          0.362
energy                                0.432
key                                       2
loudness                             -9.359
mode                                      0
speechiness                           0.112
instrumentalness                          0
liveness                              0.264
valence                               0.256
tempo                                 81.29
duration_ms                          221169
time_signature                            4
Name: 0, dtype: object


In [27]:
query_string = f'{song_ex["track_name"]} {song_ex["artist"]}'

In [31]:
from langdetect import detect # WILL NEED TO PIP INSTALL langdetect
detected_language = detect(query_string)

In [32]:
detected_language

'en'

In [33]:
type(songs)

pandas.core.frame.DataFrame

In [ ]:
if detected_language != 'en':
    # remove it from songs
    songs = gather_songs(sp,generes_list,settings_df, goal)
    data = songs.drop(labels=0, axis=0)

In [61]:
settings = [{"Name": "danceability", "On": False, "Level": 73},
            {"Name": "energy", "On": False,"Level": 88},
            {"Name": "valence", "On": False,"Level": 0},
            {"Name": "loudness","On": False, "Level": 100},
            {"Name": "instrumentalness","On": False, "Level": 22},
            {"Name": "liveness", "On": False,"Level": 10}]

settings_df = pd.DataFrame(settings)
goal = 5
generes_list = ["latin","j-dance","indian"]

songs = gather_songs(sp,generes_list,settings_df, goal)

got reccomendations
600
here 1
here 2
here 3
538
here 226
found 538


In [64]:
# OVERALL
to_remove = []
for i in range(len(songs)):
    query_string = f'{songs.iloc[i]["track_name"]} {songs.iloc[i]["artist"]}'
    detected_language = detect(query_string)
    if detected_language != 'en':
        to_remove.append(i)

songs = songs.drop(labels=to_remove, axis=0)
songs = songs.reset_index(drop=True) 

In [65]:
songs

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Arijit Singh,Love Dose Arijit Singh,"Tose Naina (From ""Mickey Virus)",2fMlezwwP5Hr1UwuAknN4o,0.523,0.561,4,-6.474,1,0.0329,0,0.109,0.31,139.885,263549,4
1,Kojo Funds,Golden Boy,Check (with RAYE),53i7ll4F1dpYS0ZUQx9USd,0.41,0.718,3,-6.295,0,0.126,0,0.079,0.57,94.332,215573,4
2,NAV,Good Intentions,Turks (with Gunna & ft. Travis Scott),7IMwEtpAiJanqF6vQteZN2,0.894,0.618,7,-4.356,0,0.129,0,0.558,0.379,119.1,161320,4
3,Internet Money,B4 The Storm,Thrusting (feat. Swae Lee & Future),0ShIftP1VFRovK53oQn8KJ,0.675,0.577,7,-5.431,0,0.338,0,0.0857,0.568,193.777,222680,4
4,Stylo G,Touch Down (Remix),Touch Down - Remix,6YvcLedQCbwEN9PWeGjK46,0.603,0.707,11,-7.413,0,0.3,0.000001,0.116,0.488,100.104,195146,4
5,Chronixx,Dread & Terrible,Here Comes Trouble,5Wwr2S7QZTR5PVJn6jhgdk,0.758,0.786,4,-4.897,0,0.108,0,0.0658,0.786,83.495,234587,4
6,Spice,Needle Eye - Single,Needle Eye,6gt4crRNZ5Soua4BP52HXv,0.655,0.745,1,-4.683,0,0.453,0,0.104,0.773,95.596,185626,4
7,Nicki Minaj,Beam Me Up Scotty,Crocodile Teeth (with Skillibeng) - Remix,2nxy7TbWDWOBm2bseZJX2A,0.635,0.634,1,-8.91,0,0.356,0.000107,0.198,0.224,107.008,217883,4
8,Ayo Jay,Your Number REMIX (feat. Chris Brown & Kid Ink),Your Number REMIX (feat. Chris Brown & Kid Ink),0v9w61bEVbIgYbVu0frJag,0.752,0.595,9,-6.763,0,0.121,0,0.291,0.746,100.021,271607,4
9,ChocQuibTown,ChocQuib House,Quisiera,4ZzJYm533TMFG7srgVldO1,0.599,0.733,10,-6.586,1,0.259,0,0.0889,0.52,89.138,232566,4
